In [ ]:
import pandas as pd
import requests, uuid

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

### Read Source Data and Drop any null comments

In [ ]:
# read data
df = pd.read_csv("Hesas Misr_coments.csv")

# select not null data
df = df[df["message"].notnull()]

# reset index
df.reset_index(drop=True, inplace=True)

#### add keys and credential

In [ ]:
## azure translation key and endpoint
translate_key = ""
translate_endpoint = ""

## text analysis key and exdpoint
Text_key = ""
Text_endpoint = ""

#### authoricate text analysis client

In [ ]:
## Authorication function
def authenticate_text_client():
    
    global Text_key
    global Text_endpoint
    
    key = Text_key
    endpoint = Text_endpoint

    text_analytics_client = TextAnalyticsClient( endpoint=endpoint,  credential= AzureKeyCredential(key))
    
    return text_analytics_client

In [ ]:
## create text analysis cliient
text_analysis_client = authenticate_text_client()

## Translate function
**This function takes sentment and translate it to Englist.**



In [ ]:
def translate(sent):
    
    global translate_key
    global translate_endpoint
    
     
    # subscription key and endpoint
    subscription_key = translate_key
    endpoint = translate_endpoint   

    # select endpoint resource url url
    constructed_url = endpoint + '/translate'

    # set request parameters and headers
    params = {
        'api-version': '3.0',
        'to': 'en',
        'toScript': 'latn'}  

    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': "eastus",
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())

    # set body {text or document to translate}.
    body = [{
        'text': sent}]

    
    # send the request and get translated comment
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    
        # convert data to json
    response = request.json()

    # return translated text
    return response[0]['translations'][0]['text']



### NER function

In [ ]:
def NER(text):
    global text_analysis_client
    
    ## add limit to the comment or it is a spam comment
    if len(text) > 750:
        return True
    
    try:
        # listing the text 
        # send request with the text
        NER_res = text_analysis_client.recognize_entities(documents = [text])[0]
        
        # if no result from request (complicated text) return False 
        try:
            # check if hte result is person return True
            # if any error its not human name
            entity = NER_res.entities[0]
            if entity.category == 'Person':
                return True
            else:
              return False
        except IndexError:
            return False
        
    except Exception as err:
        print("Encountered exception. {}".format(err))

### text analysis function 

In [ ]:
def sentiment_analysis(translated):
    
    global text_analysis_client
    

    if not(is_human):
        

        ## analyse translated text score
        response =  text_analysis_client.analyze_sentiment(documents=[translated])[0]
        ## P1 is the overall sentment 
        p1 = "Document Sentiment: {}\n".format(response.sentiment)
        ## P2 is every rate score 
        p2 = "\nOverall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
            response.confidence_scores.positive,
            response.confidence_scores.neutral,
            response.confidence_scores.negative)
        
        result = p1 + p2
        
        return result
        
    
    # if the result is human name
    # will not ranslate and assume the text is neutral
    else:
        result = "Document Sentiment: neutral\n" + "Overall scores: positive=0.00; neutral=1.00; negative=0.00\n"
        return result

#### create data dict

In [ ]:
data = {
        "text":[],
        "Translate":[],
        "is_human":[],
        "Sentiment":[]
   
        }

### starting function

the function recalll itself untill the start number is the end number

> start from number to end number or from start number until end of data 
>
> write the result to .txt/data table 

In [ ]:
def start(file, start= 0, end = -1):
    global data
    
    try: 
        if start == end:
            print("Data Limit Reached")
            return


        # load text from the data frame
        text = df.message[start]

        # checkif the comment is human mention
        is_human = NER(text)

        # translate the text to English
        translated = translate(text)

        # runsentment analysis on translated text
        result = sentiment_analysis(translated)


        ## add text to DF
        data["text"].append(text)

        ## Add humanity uesult
        file.write("\nis human: {}\n".format(is_human))
        data["is_human"].append(is_human)

        ## add Translated text 
        file.write("Translate: {}\n".format(translated))
        data["Translate"].append(translated)

        ## add analysis result
        data["Sentiment"].append(result)
        file.write(result)

        ## write Done and indexnumber 
        file.write("\n done {}\n".format(start))

        """
        Recall the Function Again until no Data or reach limit
        """
        start(file, start+1)
        
        
    except KeyError as e:
        print("No More Data\nprocessis done")
        return

### open file and start 

In [ ]:
start = int("")
end   = int("")

with open('{}{}-{}.txt'.format("Hessas",start,end-1),  'w', encoding='utf-8') as f:
    
    ## open start function
    start(f, 0)
    
# after exit fromstart function 
# save the data to .csv file and .txt  file
df = pd.DataFrame(data)
df.to_csv("{}{}-{}.csv".format("Hessas", start, end-1),index = False)